In [2]:
import tsdm
dir(tsdm)

In [2]:
import numpy as np
missing = np.random.choice([True, False], x.shape)

In [5]:
tsdm.download_model('NODE')

In [6]:
x = tsdm.load_dataset('electricity')
x

In [10]:
x = x.where(missing)
x

In [11]:
tsdm.make_dense_triplets(x)

In [12]:
sparse_x = tsdm.make_sparse_triplets(x)
sparse_x

In [13]:
x, m, d = tsdm.make_masked_format(x)
display(x, m, d)